In [1]:
import yaml
import torch.nn as nn
from collections import defaultdict, Counter
import onmt
from onmt.inputters.inputter import (
    _load_vocab,
    _build_fields_vocab,
    get_fields,
    IterOnDevice,
)
from onmt.inputters.corpus import ParallelCorpus
from onmt.inputters.dynamic_iterator import DynamicDatasetIter
from onmt.utils.misc import set_random_seed
from onmt.utils.parse import ArgumentParser
from onmt.opts import dynamic_prepare_opts
from transformers import AutoTokenizer
import torch

from onmt.utils.logging import init_logger, logger

from bert.models import BertEncoder
from bert.transforms import BertTransform

In [2]:
parser = ArgumentParser()
onmt.opts.train_opts(parser)

In [3]:
base_args = ["-config", "../opennmt/experiments-bert/runs/bert/config.yaml"]
opts, unknown = parser.parse_known_args(base_args)

In [4]:

init_logger(log_file=opts.log_file)

is_cuda = torch.cuda.is_available()
set_random_seed(opts.seed, is_cuda)

# build_vocab_main(opts)

src_vocab_path = opts.src_vocab
tgt_vocab_path = opts.tgt_vocab

counters = defaultdict(Counter)
eng_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

_src_vocab = [[k, v] for k, v in eng_tokenizer.get_vocab().items()]
_src_vocab_size = len(_src_vocab)
for k, _ in _src_vocab:
    counters["src"][k] = 1000  # Set dummy count

# load target vocab
_tgt_vocab, _tgt_vocab_size = _load_vocab(tgt_vocab_path, "tgt", counters)

# initialize fields
src_nfeats, tgt_nfeats = 0, 0  # do not support word features for now
fields = get_fields("text", src_nfeats, tgt_nfeats)

share_vocab = False
vocab_size_multiple = 1
src_vocab_size = _src_vocab_size
tgt_vocab_size = opts.tgt_vocab_size
src_words_min_frequency = 0
tgt_words_min_frequency = opts.tgt_words_min_frequency

vocab_fields = _build_fields_vocab(
    fields,
    counters,
    "text",
    share_vocab,
    vocab_size_multiple,
    src_vocab_size,
    src_words_min_frequency,
    tgt_vocab_size,
    tgt_words_min_frequency,
)

# src_text_field = vocab_fields["src"].base_field
# src_vocab = src_text_field.vocab
# src_padding = src_vocab.stoi[src_text_field.pad_token]

tgt_text_field = vocab_fields["tgt"].base_field
tgt_vocab = tgt_text_field.vocab
tgt_padding = tgt_vocab.stoi[tgt_text_field.pad_token]


[2021-10-19 14:18:52,168 INFO] Loading tgt vocabulary from /home/bana/Code/hu-nmt/opennmt/experiments-bert/vocabs/vocab.hu
[2021-10-19 14:18:52,169 INFO] Loaded tgt vocab has 32 tokens.
[2021-10-19 14:18:52,172 INFO]  * tgt vocab size: 36.
[2021-10-19 14:18:52,213 INFO]  * src vocab size: 28998.


In [5]:
encoder = BertEncoder()
decoder_embeddings = onmt.modules.Embeddings(
    opts.tgt_word_vec_size, len(tgt_vocab), word_padding_idx=tgt_padding
)
decoder = onmt.decoders.transformer.TransformerDecoder.from_opt(
    opts, decoder_embeddings
)

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = onmt.models.model.NMTModel(encoder, decoder)
model.to(device)

logger.info(model)
model.count_parameters(log=logger.info)

[2021-10-19 14:18:59,128 INFO] NMTModel(
  (encoder): BertEncoder(
    (base): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(28996, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768

(108310272, 16835584)

In [7]:
# Specify the tgt word generator and loss computation module
model.generator = nn.Sequential(
    nn.Linear(opts.dec_rnn_size, len(tgt_vocab)), nn.LogSoftmax(dim=-1)
).to(device)

optimizer = onmt.utils.optimizers.Optimizer.from_opt(model, opts)

In [8]:
data_config = yaml.load(opts.data)
opts.data = data_config
src_train = data_config["corpus"]["path_src"]
tgt_train = data_config["corpus"]["path_tgt"]
src_val = data_config["valid"]["path_src"]
tgt_val = data_config["valid"]["path_tgt"]

# build the ParallelCorpus
corpus = ParallelCorpus("corpus", src_train, tgt_train)
valid = ParallelCorpus("valid", src_val, tgt_val)

bertTransform = BertTransform(opts, src_tokenizer=eng_tokenizer)
bertTransform.warm_up()

gpu_rank = 0 if is_cuda else -1  # TODO: read from config


/tmp/ipykernel_39351/2022652398.py:1: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data_config = yaml.load(opts.data)


In [9]:
train_iter = DynamicDatasetIter.from_opts(
    corpora={"corpus": corpus},
    transforms={"berttokenizer": bertTransform},
    fields=vocab_fields,
    opts=opts,
    is_train=True,
)
train_iter = iter(IterOnDevice(train_iter, gpu_rank))
valid_iter = DynamicDatasetIter.from_opts(
    corpora={"valid": valid},
    transforms={"berttokenizer": bertTransform},
    fields=vocab_fields,
    opts=opts,
    is_train=False,
)
valid_iter = IterOnDevice(valid_iter, gpu_rank)

report_manager = onmt.utils.build_report_manager(opts, gpu_rank)
model_saver = onmt.models.build_model_saver(
    model_opt=opts, opt=opts, model=model, fields=fields, optim=optimizer
)

trainer = onmt.trainer.build_trainer(
    opts,
    device_id=gpu_rank,
    model=model,
    fields=fields,
    optim=optimizer,
    model_saver=model_saver,
)

2021-10-19 14:18:59.747479: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-19 14:18:59.747508: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
trainer.train(
    train_iter=train_iter,
    train_steps=opts.train_steps,
    valid_iter=valid_iter,
    valid_steps=opts.valid_steps,
    save_checkpoint_steps=opts.save_checkpoint_steps,
)

[2021-10-19 14:19:01,042 INFO] Start training loop and validate every 10000 steps...
[2021-10-19 14:19:01,043 INFO] corpus's transforms: TransformPipe(BertTransform(share_vocab=False, src_subword_model=/home/bana/Code/hu-nmt/opennmt/experiments-bert/sp_models/bpe_en.model, tgt_subword_model=/home/bana/Code/hu-nmt/opennmt/experiments-bert/sp_models/bpe_hu.model, src_subword_alpha=0.0, tgt_subword_alpha=0.0, src_subword_vocab=, tgt_subword_vocab=, src_vocab_threshold=0, tgt_vocab_threshold=0, src_subword_nbest=1, tgt_subword_nbest=1))
[2021-10-19 14:19:01,044 INFO] Loading ParallelCorpus(/home/bana/Data/combined-tiny/hunglish2-tiny-train.en, /home/bana/Data/combined-tiny/hunglish2-tiny-train.hu, align=None)...
[2021-10-19 14:19:01,045 INFO] Loading ParallelCorpus(/home/bana/Data/combined-tiny/hunglish2-tiny-train.en, /home/bana/Data/combined-tiny/hunglish2-tiny-train.hu, align=None)...
[2021-10-19 14:19:01,046 INFO] Loading ParallelCorpus(/home/bana/Data/combined-tiny/hunglish2-tiny-trai

RuntimeError: mat1 dim 1 must match mat2 dim 0

In [ ]:
opts

Namespace(aan_useffn=False, accum_count=[1], accum_steps=[0], adagrad_accumulator_init=0, adam_beta1=0.9, adam_beta2=0.998, alignment_heads=0, alignment_layer=-3, apex_opt_level='O1', attention_dropout=[0.1], average_decay=0.0005, average_every=1, batch_size=2, batch_size_multiple=None, batch_type='sents', bidir_edges=True, both_embeddings=None, bridge=False, bridge_extra_node=True, brnn=None, bucket_size=2048, cnn_kernel_width=3, config='../opennmt/experiments-bert/runs/bert/config.yaml', context_gate=None, copy_attn=False, copy_attn_force=False, copy_attn_type=None, copy_loss_by_seqlength=False, coverage_attn=False, data={'corpus': {'path_src': '/home/bana/Data/combined-tiny/hunglish2-tiny-train.en', 'path_tgt': '/home/bana/Data/combined-tiny/hunglish2-tiny-train.hu', 'weight': 1, 'transforms': ['berttokenizer']}, 'valid': {'path_src': '/home/bana/Data/combined-tiny/hunglish2-tiny-valid.en', 'path_tgt': '/home/bana/Data/combined-tiny/hunglish2-tiny-valid.hu', 'weight': 1, 'transforms